In [1]:
import pandas as pd
import csv
import re
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 300
pd.options.display.max_rows = 300

%matplotlib inline

In [2]:
from modules import rename_dict as labels_names

In [3]:
labels_names

{'about_whole_nation_recoded': {'level': 'ethnicity',
  'measument_level': 'nominal',
  'recode': {'0': {'label': 'no', 'value': 0},
   '1': {'label': 'yes', 'value': 1},
   'unk': {'label': 'unk', 'value': nan}},
  'rus': 'Говорит ли автор об этносе в целом?'},
 'assessor': 'Кодировщик',
 'comment': 'Комментарий кодировщика',
 'date': 'Дата кодирования',
 'do_text_make_sense_raw': {'level': 'answer',
  'measument_level': 'nominal',
  'recode': {'joke': {'label': 'no', 'value': 0},
   'lang': {'label': 'no', 'value': 0},
   'no': {'label': 'no', 'value': 0},
   'yes': {'label': 'yes', 'value': 1}},
  'rus': 'Понятно ли Вам о чём текст?'},
 'document.id': 'ID документа',
 'encourage_aggression_meaning': {'level': 'ethnicity',
  'measument_level': 'ordinal',
  'recode': {'no': {'label': 'no', 'value': 0},
   'yes latent': {'label': 'yes latent', 'value': 1},
   'yes open': {'label': 'yes open', 'value': 2}},
  'rus': 'Призывает ли автор к насильственному оффлайновому действию по отношени

Заменил табуляция по регулярке `[^"]\t[^"]`.

In [4]:
meta = pd.read_table(
    "../data/file1_14998_texts_and_metadata.txt", encoding="UTF-8", sep="\t", parse_dates=["date"])

In [5]:
meta.head()

,document.id,source,stage,source_text,lemm_text,city,region,date
0,885072939,iqbuzz,2,"[[id178370173|Mia], но у вас же бред написан. ...",mia бред написать какой русский вообще грузин ...,Мурманск,Мурманская область,2015-05-05
1,627169733,iqbuzz,2,Для примера приведу по датам ключевые моменты ...,пример привести дата ключевой момент война хро...,Хабаровск,Хабаровский край,2014-10-13
2,632784345,iqbuzz,2,Фонарь!!!! Сбербанк России надо переименовать ...,фонарь сбербанк россия переименовать сбербанк ...,Сочи,Краснодарский край,2014-10-20
3,632886756,iqbuzz,2,"Олег, дак какая война у нас кроме чеченской бы...",олег дак какой война кроме чеченский про говор...,Москва,Москва,2014-10-20
4,635041456,iqbuzz,2,ДНР. Донецк. Интервью с ополченцем. Позывной \...,донецк интервью ополченец позывной молодая опо...,Москва,Москва,2014-10-23


In [6]:
meta.shape

(14998, 8)

In [7]:
meta.dtypes

document.id             int64
source                 object
stage                   int64
source_text            object
lemm_text              object
city                   object
region                 object
date           datetime64[ns]
dtype: object

In [8]:
labels = pd.read_table(
    "../data/file2_14998_coding_results.txt", encoding="UTF-8", sep="\t")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
labels.rename(columns={"data": "date"}, inplace=True)

In [10]:
labels.duplicated().sum()

43

In [11]:
labels = labels[~labels.duplicated()]

In [12]:
labels.duplicated().sum()

0

In [13]:
labels["for_questions_about_text"].value_counts()

1    46342
0    38399
Name: for_questions_about_text, dtype: int64

In [14]:
labels[labels["for_questions_about_text"] == 0]

,document.id,source,stage,date,assessor,seed_eth_group,for_questions_about_text,do_text_make_sense_raw,do_text_make_sense_recoded,has_ethnonym_raw,has_ethnonym_recoded,is_text_neg_recoded,is_text_positive_recoded,has_eth_conflict_recoded,has_pos_eth_interaction_recoded,has_topic_culture,has_topic_economics,has_topic_ethicity,has_topic_history,has_topic_humour,has_topic_daily_routine,has_topic_migration,has_topic_other,has_topic_politics,has_topic_religion,has_topic_society_social,eth_group_to_code,eth_group_to_code_raw,is_eth_group_to_code_main_eth_group,is_eth_group_to_code_postsoviet,about_whole_nation_recoded,opinion_about_ethnonym_recoded,encourage_aggression_recoded,represent_ethicity_recoded,is_ethicity_superior_recoded,is_ethicity_aggressor_recoded,is_ethicity_dangerous_recoded,is_duplicate_text_by_id,is_text_neg_raw,is_text_positive_raw,has_eth_conflict_raw,has_pos_eth_interaction_raw,about_whole_nation_raw,opinion_about_ethnonym_raw,encourage_aggression_raw,encourage_aggression_meaning,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
1,883,vk,1,2016-09-25 20:06:34,skuchilina,монгол,0,yes,1,several,2,0,1,0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,0.0,0.0,0,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
3,883,vk,1,2016-09-26 17:08:13,dianasadr,монгол,0,yes,1,several,2,0,2,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,1.0,0.0,0,unk,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
9,9333,vk,1,2016-07-10 22:58:37,Dasha,калмык,0,yes,1,one,1,0,1,0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,калмык,калмычка,1.0,1,0,0.0,0.0,0,unk,unk,0,no,no,weak,no,no,1.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,9333
27,22019,vk,1,2016-09-26 12:50:38,skuchilina,великоросс,0,yes,1,several,2,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,великоросс,великоросс,1.0,1,1,0.0,0.0,unk,unk,unk,0,no,no,no,no,no,2.0,2.0,1.0,no,2.0,unk,3.0,irrel,3.0,irrel,1.0,no,Другой - латгальцы,22019
71,50060,vk,1,2016-10-18 13:00:08,dianasadr,хальмг,0,yes,1,several,2,0,2,0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,хальмг,хальмг,1.0,1,1,1.0,0.0,1,1,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,3.0,yes,2.0,high,3.0,irrel,1.0,no,NaN,50060
72,50060,vk,1,2016-10-18 13:00:08,dianasadr,хальмг,0,yes,1,several,2,0,2,0,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,швед,швед,0.0,0,1,0.0,0.0,0,unk,0,0,no,no,strong,no,no,2.0,2.0,1.0,no,1.0,no,3.0,irrel,1.0,victim,1.0,no,NaN,50060
78,55368,vk,1,2016-07-01 13:00:11,yadviga,татарин,0,yes,1,several,2,-2,0,1,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,русский,русский,0.0,1,0,0.0,0.0,0,unk,unk,0,no,strong,no,yes,no,1.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,55368
86,58073,vk,1,2016-08-02 22:35:19,Tatiana,россиянин,0,yes,1,several,2,0,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,россиянин,россиянин,1.0,1,0,0.0,0.0,0,unk,unk,0,no,no,no,no,yes,1.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,58073
88,58073,vk,1,2016-06-30 12:39:57,snikolenko,россиянин,0,yes,1,several,2,0,1,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,россиянин,россиянин,1.0,1,1,0.0,0.0,unk,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,2.0,unk,3.0,irrel,3.0,irrel,1.0,no,NaN,58073
92,61238,vk,1,2016-07-15 18:41:35,Dasha,чеченец,0,yes,1,one,1,unk,unk,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,чеченец,чеченец,1.0,1,0,0.0,0.0,0,unk,unk,0,no,unk,unk,yes,no,1.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,61238


In [15]:
labels[[
    "document.id", "assessor", "eth_group_to_code",
#     "seed_eth_group"
]].duplicated().sum()

1382

In [16]:
labels[[
    "document.id", "assessor", "eth_group_to_code",
#     "seed_eth_group",
    "for_questions_about_text"
]].duplicated().sum()

1020

In [17]:
labels.duplicated(subset=[
    label for label in labels.columns.tolist()
    if label != "for_questions_about_text"
]).sum()

47

Только 47 документов совпадают, если исключить `for_questions_about_text`.

In [18]:
labels[~labels[[
    "document.id", "assessor", "eth_group_to_code",
#     "seed_eth_group"
]].duplicated()]

,document.id,source,stage,date,assessor,seed_eth_group,for_questions_about_text,do_text_make_sense_raw,do_text_make_sense_recoded,has_ethnonym_raw,has_ethnonym_recoded,is_text_neg_recoded,is_text_positive_recoded,has_eth_conflict_recoded,has_pos_eth_interaction_recoded,has_topic_culture,has_topic_economics,has_topic_ethicity,has_topic_history,has_topic_humour,has_topic_daily_routine,has_topic_migration,has_topic_other,has_topic_politics,has_topic_religion,has_topic_society_social,eth_group_to_code,eth_group_to_code_raw,is_eth_group_to_code_main_eth_group,is_eth_group_to_code_postsoviet,about_whole_nation_recoded,opinion_about_ethnonym_recoded,encourage_aggression_recoded,represent_ethicity_recoded,is_ethicity_superior_recoded,is_ethicity_aggressor_recoded,is_ethicity_dangerous_recoded,is_duplicate_text_by_id,is_text_neg_raw,is_text_positive_raw,has_eth_conflict_raw,has_pos_eth_interaction_raw,about_whole_nation_raw,opinion_about_ethnonym_raw,encourage_aggression_raw,encourage_aggression_meaning,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
0,883,vk,1,2016-09-25 20:06:34,skuchilina,монгол,1,yes,1,several,2,0,1,0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,тибетец,тибетец,0.0,0,1,0.0,0.0,0,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
1,883,vk,1,2016-09-25 20:06:34,skuchilina,монгол,0,yes,1,several,2,0,1,0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,0.0,0.0,0,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
2,883,vk,1,2016-09-26 17:08:13,dianasadr,монгол,1,yes,1,several,2,0,2,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,тибетец,тибетец,0.0,0,1,1.0,0.0,0,unk,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
3,883,vk,1,2016-09-26 17:08:13,dianasadr,монгол,0,yes,1,several,2,0,2,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,1.0,0.0,0,unk,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
4,883,vk,1,2016-07-29 15:10:42,Oleg Nagornyy,монгол,1,yes,1,one,1,-2,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,strong,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,883
5,2942,vk,1,2016-08-01 09:25:19,Oleg Nagornyy,помор,1,yes,1,one,1,0,2,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,strong,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2942
6,2942,vk,1,2016-09-23 01:14:58,nicketas,помор,1,yes,1,no,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2942
7,2942,vk,1,2016-08-03 09:33:54,Tatiana,помор,1,yes,1,no,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2942
8,9333,vk,1,2016-07-10 22:58:24,Dasha,калмык,1,yes,1,one,1,0,1,0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,калмык,калмычка,1.0,1,0,0.0,0.0,0,unk,unk,0,no,no,weak,no,no,1.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,9333
10,9333,vk,1,2016-08-01 12:53:10,Oleg Nagornyy,калмык,1,yes,1,one,1,0,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,strong,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9333


In [19]:
labels.head()

,document.id,source,stage,date,assessor,seed_eth_group,for_questions_about_text,do_text_make_sense_raw,do_text_make_sense_recoded,has_ethnonym_raw,has_ethnonym_recoded,is_text_neg_recoded,is_text_positive_recoded,has_eth_conflict_recoded,has_pos_eth_interaction_recoded,has_topic_culture,has_topic_economics,has_topic_ethicity,has_topic_history,has_topic_humour,has_topic_daily_routine,has_topic_migration,has_topic_other,has_topic_politics,has_topic_religion,has_topic_society_social,eth_group_to_code,eth_group_to_code_raw,is_eth_group_to_code_main_eth_group,is_eth_group_to_code_postsoviet,about_whole_nation_recoded,opinion_about_ethnonym_recoded,encourage_aggression_recoded,represent_ethicity_recoded,is_ethicity_superior_recoded,is_ethicity_aggressor_recoded,is_ethicity_dangerous_recoded,is_duplicate_text_by_id,is_text_neg_raw,is_text_positive_raw,has_eth_conflict_raw,has_pos_eth_interaction_raw,about_whole_nation_raw,opinion_about_ethnonym_raw,encourage_aggression_raw,encourage_aggression_meaning,represent_ethicity_raw,represent_ethicity_meaning,is_ethicity_superior_raw,is_ethicity_superior_meaning,is_ethicity_aggressor_raw,is_ethicity_aggressor_meaning,is_ethicity_dangerous_raw,is_ethicity_dangerous_meaning,comment,old_id
0,883,vk,1,2016-09-25 20:06:34,skuchilina,монгол,1,yes,1,several,2,0,1,0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,тибетец,тибетец,0.0,0,1,0.0,0.0,0,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
1,883,vk,1,2016-09-25 20:06:34,skuchilina,монгол,0,yes,1,several,2,0,1,0,1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,0.0,0.0,0,unk,unk,0,no,no,weak,no,yes,2.0,2.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
2,883,vk,1,2016-09-26 17:08:13,dianasadr,монгол,1,yes,1,several,2,0,2,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,тибетец,тибетец,0.0,0,1,1.0,0.0,0,unk,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
3,883,vk,1,2016-09-26 17:08:13,dianasadr,монгол,0,yes,1,several,2,0,2,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,монгол,монгол,1.0,1,1,1.0,0.0,0,unk,unk,0,no,no,strong,no,no,2.0,1.0,1.0,no,1.0,no,3.0,irrel,3.0,irrel,1.0,no,NaN,883
4,883,vk,1,2016-07-29 15:10:42,Oleg Nagornyy,монгол,1,yes,1,one,1,-2,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,strong,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,883


In [20]:
labels.columns

Index(['document.id', 'source', 'stage', 'date', 'assessor', 'seed_eth_group',
       'for_questions_about_text', 'do_text_make_sense_raw',
       'do_text_make_sense_recoded', 'has_ethnonym_raw',
       'has_ethnonym_recoded', 'is_text_neg_recoded',
       'is_text_positive_recoded', 'has_eth_conflict_recoded',
       'has_pos_eth_interaction_recoded', 'has_topic_culture',
       'has_topic_economics', 'has_topic_ethicity', 'has_topic_history',
       'has_topic_humour', 'has_topic_daily_routine', 'has_topic_migration',
       'has_topic_other', 'has_topic_politics', 'has_topic_religion',
       'has_topic_society_social', 'eth_group_to_code',
       'eth_group_to_code_raw', 'is_eth_group_to_code_main_eth_group',
       'is_eth_group_to_code_postsoviet', 'about_whole_nation_recoded',
       'opinion_about_ethnonym_recoded', 'encourage_aggression_recoded',
       'represent_ethicity_recoded', 'is_ethicity_superior_recoded',
       'is_ethicity_aggressor_recoded', 'is_ethicity_dangerous

In [21]:
sorted(labels.columns.tolist())

['about_whole_nation_raw',
 'about_whole_nation_recoded',
 'assessor',
 'comment',
 'date',
 'do_text_make_sense_raw',
 'do_text_make_sense_recoded',
 'document.id',
 'encourage_aggression_meaning',
 'encourage_aggression_raw',
 'encourage_aggression_recoded',
 'eth_group_to_code',
 'eth_group_to_code_raw',
 'for_questions_about_text',
 'has_eth_conflict_raw',
 'has_eth_conflict_recoded',
 'has_ethnonym_raw',
 'has_ethnonym_recoded',
 'has_pos_eth_interaction_raw',
 'has_pos_eth_interaction_recoded',
 'has_topic_culture',
 'has_topic_daily_routine',
 'has_topic_economics',
 'has_topic_ethicity',
 'has_topic_history',
 'has_topic_humour',
 'has_topic_migration',
 'has_topic_other',
 'has_topic_politics',
 'has_topic_religion',
 'has_topic_society_social',
 'is_duplicate_text_by_id',
 'is_eth_group_to_code_main_eth_group',
 'is_eth_group_to_code_postsoviet',
 'is_ethicity_aggressor_meaning',
 'is_ethicity_aggressor_raw',
 'is_ethicity_aggressor_recoded',
 'is_ethicity_dangerous_meaning',

In [22]:
for column in sorted(labels.columns.tolist()):
    u_vals = labels[column].unique().tolist()
    vals_counts = []
    if len(u_vals) > 15:
        u_vals = "Много значений"
    else:
        for u_val in u_vals:
            s = (labels[column] == u_val).sum()
            vals_counts.append((s, u_val))
        u_vals = ", ".join(["**{}** ({})".format(val, counts) for counts, val in vals_counts])
    
    print("* `{}`:\t{}".format(str(column), u_vals))

* `about_whole_nation_raw`:	**2.0** (51448), **nan** (0), **1.0** (9638), **3.0** (2109)
* `about_whole_nation_recoded`:	**1** (51448), **nan** (0), **0** (9638), **unk** (2109)
* `assessor`:	Много значений
* `comment`:	Много значений
* `date`:	Много значений
* `do_text_make_sense_raw`:	**yes** (78340), **no** (4331), **lang** (1883), **joke** (187)
* `do_text_make_sense_recoded`:	**1** (78340), **0** (6401)
* `document.id`:	Много значений
* `encourage_aggression_meaning`:	**no** (62541), **nan** (0), **yes open** (221), **yes latent** (433)
* `encourage_aggression_raw`:	**1.0** (62541), **nan** (0), **3.0** (221), **2.0** (433)
* `encourage_aggression_recoded`:	**0.0** (62541), **nan** (0), **2.0** (221), **1.0** (433)
* `eth_group_to_code`:	Много значений
* `eth_group_to_code_raw`:	Много значений
* `for_questions_about_text`:	**1** (46342), **0** (38399)
* `has_eth_conflict_raw`:	**no** (52411), **yes** (21465), **unk** (4464), **nan** (0)
* `has_eth_conflict_recoded`:	**0** (52411),

In [23]:
labels.shape

(84741, 56)

In [24]:
labels.isnull().sum()

document.id                                0
source                                     0
stage                                      0
date                                       0
assessor                                   4
seed_eth_group                             0
for_questions_about_text                   0
do_text_make_sense_raw                     0
do_text_make_sense_recoded                 0
has_ethnonym_raw                        6401
has_ethnonym_recoded                    6401
is_text_neg_recoded                     6401
is_text_positive_recoded                6401
has_eth_conflict_recoded                6401
has_pos_eth_interaction_recoded         6401
has_topic_culture                      11522
has_topic_economics                    11522
has_topic_ethicity                     11522
has_topic_history                      11522
has_topic_humour                       11522
has_topic_daily_routine                11522
has_topic_migration                    11522
has_topic_

In [25]:
labels.dtypes

document.id                              int64
source                                  object
stage                                    int64
date                                    object
assessor                                object
seed_eth_group                          object
for_questions_about_text                 int64
do_text_make_sense_raw                  object
do_text_make_sense_recoded               int64
has_ethnonym_raw                        object
has_ethnonym_recoded                    object
is_text_neg_recoded                     object
is_text_positive_recoded                object
has_eth_conflict_recoded                object
has_pos_eth_interaction_recoded         object
has_topic_culture                      float64
has_topic_economics                    float64
has_topic_ethicity                     float64
has_topic_history                      float64
has_topic_humour                       float64
has_topic_daily_routine                float64
has_topic_mig

In [26]:
len(labels["document.id"].unique()), len(meta["document.id"].unique())

(14591, 14591)

* `about_whole_nation_raw`:	**2.0** (51473), **nan** (0), **1.0** (9645), **3.0** (2109)
* `about_whole_nation_recoded`:	**1** (51473), **nan** (0), **0** (9645), **unk** (2109)
* `assessor`:	Много значений
* `comment`:	Много значений
* `data`:	Много значений
* `do_text_make_sense_raw`:	**yes** (78372), **no** (4334), **lang** (1890), **joke** (188)
* `do_text_make_sense_recoded`:	**1** (78372), **0** (6412)
* `document.id`:	Много значений
* `encourage_aggression_meaning`:	**no** (62570), **nan** (0), **yes open** (221), **yes latent** (436)
* `encourage_aggression_raw`:	**1.0** (62570), **nan** (0), **3.0** (221), **2.0** (436)
* `encourage_aggression_recoded`:	**0.0** (62570), **nan** (0), **2.0** (221), **1.0** (436)
* `eth_group_to_code`:	Много значений
* `eth_group_to_code_raw`:	Много значений
* `for_questions_about_text`:	**1** (46342), **0** (38442)
* `has_eth_conflict_raw`:	**no** (52419), **yes** (21487), **unk** (4466), **nan** (0)
* `has_eth_conflict_recoded`:	**0** (52419), **1** (21487), **unk** (4466), **nan** (0)
* `has_ethnonym_raw`:	**several** (56697), **one** (16155), **no** (5121), **nan** (0), **unk** (399)
* `has_ethnonym_recoded`:	**2** (56697), **1** (16155), **0** (5121), **nan** (0), **unk** (399)
* `has_pos_eth_interaction_raw`:	**yes** (13603), **no** (62783), **nan** (0), **unk** (1986)
* `has_pos_eth_interaction_recoded`:	**1** (13603), **0** (62783), **nan** (0), **unk** (1986)
* `has_topic_culture`:	**1.0** (6558), **0.0** (66693), **nan** (0)
* `has_topic_daily_routine`:	**0.0** (67706), **1.0** (5545), **nan** (0)
* `has_topic_economics`:	**0.0** (71671), **nan** (0), **1.0** (1580)
* `has_topic_ethicity`:	**1.0** (63227), **0.0** (10024), **nan** (0)
* `has_topic_history`:	**0.0** (59943), **nan** (0), **1.0** (13308)
* `has_topic_humour`:	**0.0** (72264), **nan** (0), **1.0** (987)
* `has_topic_migration`:	**0.0** (70809), **nan** (0), **1.0** (2442)
* `has_topic_other`:	**0.0** (67454), **nan** (0), **1.0** (5797)
* `has_topic_politics`:	**0.0** (56469), **nan** (0), **1.0** (16782)
* `has_topic_religion`:	**0.0** (70993), **nan** (0), **1.0** (2258)
* `has_topic_society_social`:	**0.0** (56902), **nan** (0), **1.0** (16349)
* `is_duplicate_text_by_id`:	**no** (84107), **yes** (677)
* **???**`is_eth_group_to_code_main_eth_group`:	**0.0** (38185), **1.0** (25042), **nan** (0)
* **???**`is_eth_group_to_code_postsoviet`:	**0** (4493), **1** (57091), **nan** (0), **unk** (1643)
* `is_ethicity_aggressor_meaning`:	**irrel** (51612), **nan** (0), **agressor** (5069), **victim** (6546)
* `is_ethicity_aggressor_raw`:	**3.0** (51612), **nan** (0), **2.0** (5069), **1.0** (6546)
* `is_ethicity_aggressor_recoded`:	**unk** (51612), **nan** (0), **1** (5069), **0** (6546)
* `is_ethicity_dangerous_meaning`:	**no** (57413), **nan** (0), **yes** (3809), **irrel** (2005)
* `is_ethicity_dangerous_raw`:	**1.0** (57413), **nan** (0), **2.0** (3809), **3.0** (2005)
* `is_ethicity_dangerous_recoded`:	**0** (57413), **nan** (0), **1** (3809), **unk** (2005)
* `is_ethicity_superior_meaning`:	**irrel** (50701), **nan** (0), **high** (4877), **low** (7649)
* `is_ethicity_superior_raw`:	**3.0** (50701), **nan** (0), **2.0** (4877), **1.0** (7649)
* `is_ethicity_superior_recoded`:	**unk** (50701), **nan** (0), **1** (4877), **0** (7649)
* `is_text_neg_raw`:	**no** (36549), **strong** (8686), **weak** (29016), **nan** (0), **unk** (4121)
* `is_text_neg_recoded`:	**0** (36549), **-2** (8686), **-1** (29016), **nan** (0), **unk** (4121)
* `is_text_positive_raw`:	**weak** (12148), **strong** (2580), **no** (61360), **nan** (0), **unk** (2284)
* `is_text_positive_recoded`:	**1** (12148), **2** (2580), **0** (61360), **nan** (0), **unk** (2284)
* `old_id`:	Много значений
* `opinion_about_ethnonym_raw`:	**2.0** (43306), **1.0** (10654), **nan** (0), **3.0** (9267)
* `opinion_about_ethnonym_recoded`:	**0.0** (43306), **1.0** (10654), **nan** (0), **-1.0** (9267)
* `represent_ethicity_meaning`:	**no** (38160), **nan** (0), **unk** (18870), **yes** (6197)
* `represent_ethicity_raw`:	**1.0** (38160), **nan** (0), **2.0** (18870), **3.0** (6197)
* `represent_ethicity_recoded`:	**0** (38160), **nan** (0), **unk** (18870), **1** (6197)
* `seed_eth_group`:	Много значений
* `source`:	**vk** (7794), **iqbuzz** (76990)
* `stage`:	**1** (7794), **3** (43413), **2** (33577)

In [27]:
len(labels["seed_eth_group"].unique()), labels["seed_eth_group"].value_counts()

(123, киргиз              1846
 азербайджанец       1566
 украинец            1564
 туркмен             1402
 караим              1379
 россиянин           1348
 удмурт              1299
 мордва              1279
 гагауз              1242
 балкарец            1194
 хакас               1180
 чуваш               1169
 жид                 1146
 калмык              1118
 эвенк               1112
 вепс                1102
 башкир              1098
 манси               1096
 карачаевец          1080
 кумык               1061
 даргинец            1025
 малорос             1025
 уйгур               1013
 адыг                1008
 аварец              1000
 угр                  982
 среднеазиат          973
 ненец                972
 черкес               970
 эрзя                 969
 вайнах               945
 казах                939
 беларус              935
 нацмен               930
 абхаз                928
 молдаванин           927
 карел                925
 нанаец               923
 ингуш 

Среднее количество кодирований текстов:

Каждый текст
 * размечен различными кодировщиками
 * может включать различные этничности
 
Надо разделить переменные на text-level и ethnicity-level. 

Для каждой переменной нужно посчитать сходимость кодировщиков. Для text-level просто сходимость по кодировщикам по одному тексту. Для ethnicity-level — сходимость кодировщиков по одному тексту и одной этничности.

Убираем сомнительных кодировщиков — с очень малым количеством кодирований и Николенко.

In [28]:
blacklist = ["snikolenko", "olessia.koltsova", "yulia", "Олег"]
labels = labels[~labels["assessor"].isin(blacklist)]

In [29]:
labels["is_ethicity_superior_meaning"].replace({'no': 0, 'high': 2, 'low': 1})

0        irrel
1        irrel
2        irrel
3        irrel
4          NaN
5          NaN
6          NaN
7          NaN
8        irrel
9        irrel
10         NaN
11         NaN
12       irrel
13         NaN
14           2
15       irrel
16           1
17       irrel
18       irrel
19       irrel
20         NaN
21       irrel
22       irrel
23       irrel
24       irrel
25         NaN
26       irrel
27       irrel
28         NaN
29         NaN
30         NaN
31         NaN
32         NaN
33         NaN
34         NaN
35         NaN
36         NaN
37         NaN
38         NaN
39         NaN
40         NaN
41         NaN
42         NaN
43         NaN
44         NaN
45         NaN
46         NaN
47         NaN
48         NaN
49         NaN
50         NaN
51         NaN
52         NaN
53         NaN
54       irrel
55         NaN
56         NaN
57         NaN
58         NaN
59         NaN
60         NaN
61         NaN
62         NaN
63         NaN
64         NaN
65       irrel
66       i

In [30]:
df = pd.DataFrame()

for var in labels_names:
    if (type(labels_names[var]) == dict):
        values_to_replace = {
            k: v["value"]
            for k, v in labels_names[var]["recode"].items()
        }
        labels_to_replace = {
            k: v["label"]
            for k, v in labels_names[var]["recode"].items()
        }
        df[var + "_value"] = labels[var].replace(values_to_replace)
        df[var + "_label"] = labels[var].replace(labels_to_replace)
    else:
        print(var, end="\r")
        df[var] = labels[var].copy()

about_whole_nation_recoded
{'1': 1, '0': 0, 'unk': nan}
do_text_make_sense_raw
{'yes': 1, 'no': 0, 'lang': 0, 'joke': 0}
encourage_aggression_meaning
{'yes latent': 1, 'no': 0, 'yes open': 2}
has_eth_conflict_raw
{'unk': nan, 'no': 0, 'yes': 1}
has_ethnonym_raw
{'unk': nan, 'no': 0, 'one': 1, 'several': 2}
has_pos_eth_interaction_raw
{'unk': nan, 'no': 0, 'yes': 1}
has_topic_culture
{1.0: 1, 0.0: 0}
has_topic_daily_routine
{1.0: 1, 0.0: 0}
has_topic_economics
{1.0: 1, 0.0: 0}
has_topic_ethicity
{1.0: 1, 0.0: 0}
has_topic_history
{1.0: 1, 0.0: 0}
has_topic_humour
{1.0: 1, 0.0: 0}
has_topic_migration
{1.0: 1, 0.0: 0}
has_topic_other
{1.0: 1, 0.0: 0}
has_topic_politics
{1.0: 1, 0.0: 0}
has_topic_religion
{1.0: 1, 0.0: 0}
has_topic_society_social
{1.0: 1, 0.0: 0}
is_ethicity_aggressor_meaning
{'irrel': nan, 'agressor': 1, 'victim': 0}
is_ethicity_dangerous_meaning
{'no': 0, 'yes': 1, 'irrel': nan}
is_ethicity_superior_meaning
{'no': 0, 'high': 2, 'low': 1, 'irrel': nan}
is_text_neg_raw
{'n

In [31]:
df.head()

,about_whole_nation_recoded_value,about_whole_nation_recoded_label,assessor,comment,date,document.id,do_text_make_sense_raw_value,do_text_make_sense_raw_label,encourage_aggression_meaning_value,encourage_aggression_meaning_label,eth_group_to_code,has_eth_conflict_raw_value,has_eth_conflict_raw_label,has_ethnonym_raw_value,has_ethnonym_raw_label,has_pos_eth_interaction_raw_value,has_pos_eth_interaction_raw_label,has_topic_culture_value,has_topic_culture_label,has_topic_daily_routine_value,has_topic_daily_routine_label,has_topic_economics_value,has_topic_economics_label,has_topic_ethicity_value,has_topic_ethicity_label,has_topic_history_value,has_topic_history_label,has_topic_humour_value,has_topic_humour_label,has_topic_migration_value,has_topic_migration_label,has_topic_other_value,has_topic_other_label,has_topic_politics_value,has_topic_politics_label,has_topic_religion_value,has_topic_religion_label,has_topic_society_social_value,has_topic_society_social_label,is_ethicity_aggressor_meaning_value,is_ethicity_aggressor_meaning_label,is_ethicity_dangerous_meaning_value,is_ethicity_dangerous_meaning_label,is_ethicity_superior_meaning_value,is_ethicity_superior_meaning_label,is_text_neg_raw_value,is_text_neg_raw_label,is_text_positive_raw_value,is_text_positive_raw_label,opinion_about_ethnonym_recoded_value,opinion_about_ethnonym_recoded_label,represent_ethicity_meaning_value,represent_ethicity_meaning_label,seed_eth_group,source,stage
0,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1
1,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1
2,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1
3,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1
4,NaN,NaN,Oleg Nagornyy,NaN,2016-07-29 15:10:42,883,1,yes,NaN,NaN,NaN,0.0,no,1.0,one,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,NaN,NaN,NaN,NaN,NaN,2.0,strong,0.0,no,NaN,NaN,NaN,NaN,монгол,vk,1


In [32]:
df.to_msgpack("../data/coded_results.msgpack")

In [33]:
meta.head()

,document.id,source,stage,source_text,lemm_text,city,region,date
0,885072939,iqbuzz,2,"[[id178370173|Mia], но у вас же бред написан. ...",mia бред написать какой русский вообще грузин ...,Мурманск,Мурманская область,2015-05-05
1,627169733,iqbuzz,2,Для примера приведу по датам ключевые моменты ...,пример привести дата ключевой момент война хро...,Хабаровск,Хабаровский край,2014-10-13
2,632784345,iqbuzz,2,Фонарь!!!! Сбербанк России надо переименовать ...,фонарь сбербанк россия переименовать сбербанк ...,Сочи,Краснодарский край,2014-10-20
3,632886756,iqbuzz,2,"Олег, дак какая война у нас кроме чеченской бы...",олег дак какой война кроме чеченский про говор...,Москва,Москва,2014-10-20
4,635041456,iqbuzz,2,ДНР. Донецк. Интервью с ополченцем. Позывной \...,донецк интервью ополченец позывной молодая опо...,Москва,Москва,2014-10-23


In [34]:
meta.duplicated(subset=["document.id"]).sum()

407

In [35]:
meta.duplicated(subset=["document.id", "stage"]).sum()

0

In [36]:
meta.rename(columns={"date": "created_date"}, inplace=True)

In [37]:
full_df = pd.merge(df, meta, on=["document.id", "stage"])

In [38]:
full_df.shape, df.shape, meta.shape

((84573, 62), (84573, 56), (14998, 8))

In [39]:
set(meta["document.id"].unique()) - set(df["document.id"].unique())

{4234373}

Есть дубликаты в текстах засчёт того, что на 2 и 3 стадии кодирования встречаются одинаковые тексты.

In [40]:
full_df.shape, df.shape, meta.shape

((84573, 62), (84573, 56), (14998, 8))

In [41]:
full_df["eth_group_to_code"].isnull().sum()

21415

In [42]:
full_df["eth_group_to_code"].fillna(value=full_df["seed_eth_group"], inplace=True)

In [43]:
full_df["eth_group_to_code"].isnull().sum()

0

In [44]:
full_df["instance_id"] = full_df[['document.id', 'eth_group_to_code']]\
    .apply(lambda x: '_'.join(x.astype(str)), axis=1)

In [45]:
full_df.head()

,about_whole_nation_recoded_value,about_whole_nation_recoded_label,assessor,comment,date,document.id,do_text_make_sense_raw_value,do_text_make_sense_raw_label,encourage_aggression_meaning_value,encourage_aggression_meaning_label,eth_group_to_code,has_eth_conflict_raw_value,has_eth_conflict_raw_label,has_ethnonym_raw_value,has_ethnonym_raw_label,has_pos_eth_interaction_raw_value,has_pos_eth_interaction_raw_label,has_topic_culture_value,has_topic_culture_label,has_topic_daily_routine_value,has_topic_daily_routine_label,has_topic_economics_value,has_topic_economics_label,has_topic_ethicity_value,has_topic_ethicity_label,has_topic_history_value,has_topic_history_label,has_topic_humour_value,has_topic_humour_label,has_topic_migration_value,has_topic_migration_label,has_topic_other_value,has_topic_other_label,has_topic_politics_value,has_topic_politics_label,has_topic_religion_value,has_topic_religion_label,has_topic_society_social_value,has_topic_society_social_label,is_ethicity_aggressor_meaning_value,is_ethicity_aggressor_meaning_label,is_ethicity_dangerous_meaning_value,is_ethicity_dangerous_meaning_label,is_ethicity_superior_meaning_value,is_ethicity_superior_meaning_label,is_text_neg_raw_value,is_text_neg_raw_label,is_text_positive_raw_value,is_text_positive_raw_label,opinion_about_ethnonym_recoded_value,opinion_about_ethnonym_recoded_label,represent_ethicity_meaning_value,represent_ethicity_meaning_label,seed_eth_group,source_x,stage,source_y,source_text,lemm_text,city,region,created_date,instance_id
0,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_тибетец
1,1.0,yes,skuchilina,NaN,2016-09-25 20:06:34,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,1.0,yes,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,1.0,weak,1.0,neutral,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол
2,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,тибетец,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_тибетец
3,1.0,yes,dianasadr,NaN,2016-09-26 17:08:13,883,1,yes,0.0,no,монгол,0.0,no,2.0,several,0.0,no,1.0,yes,0.0,no,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,irrelevant,0.0,no,NaN,irrel,0.0,no,2.0,strong,2.0,positive,0.0,no,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол
4,NaN,NaN,Oleg Nagornyy,NaN,2016-07-29 15:10:42,883,1,yes,NaN,NaN,монгол,0.0,no,1.0,one,0.0,no,1.0,yes,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,0.0,no,NaN,NaN,NaN,NaN,NaN,NaN,2.0,strong,0.0,no,NaN,NaN,NaN,NaN,монгол,vk,1,vk,Музыка Китая – одна из древнейших в мировой му...,музыка китай древний мирова музыкальный культу...,Владимир,Владимирская область,2015-08-13,883_монгол


In [46]:
full_df.to_msgpack("../data/full_df.msgpack", compress="zlib")